In [2]:
import torch
import clip
from PIL import Image

device = "cuda" if torch.cuda.is_available() else "cpu"
# device = "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

# 定义猫的品种和详细特征描述
cat_types = [
    # 常见纯种猫
    ("暹罗猫", "a Siamese cat with cream-colored body, dark face mask, ears, legs and tail, striking blue almond-shaped eyes, and angular face"),
    ("英短蓝猫", "a British Shorthair cat with dense plush blue-gray fur, round face, chubby cheeks, copper-colored eyes, and stocky body"),
    ("波斯猫", "a Persian cat with extremely long silky fur, round face with snub nose, small ears, large round eyes, and thick ruff around neck"),
    ("缅因猫", "a Maine Coon cat with long shaggy fur, large tufted ears, bushy tail, rectangular body, and distinctive ruff around neck like a lion's mane"),
    ("布偶猫", "a Ragdoll cat with color point coat, striking blue eyes, silky semi-long fur, white paws, and large muscular body"),
    ("折耳猫", "a Scottish Fold cat with folded ears lying close to head, round face, large round eyes, and medium-sized sturdy body"),
    ("斯芬克斯", "a Sphynx cat with hairless wrinkled skin, large bat-like ears, prominent cheekbones, and muscular body"),
    ("孟加拉豹猫", "a Bengal cat with distinctive leopard-like spots or marbled pattern, muscular body, high contrast coat, and wild appearance"),
    ("俄罗斯蓝猫", "a Russian Blue cat with plush double coat in silver-blue color, green almond-shaped eyes, and graceful body"),
    
    # 东方品种
    ("暹罗短毛猫", "an Oriental Shorthair cat with sleek coat, large pointed ears, almond-shaped eyes, and long slender body"),
    ("巴厘猫", "a Balinese cat with long silky point-colored coat, blue eyes, long elegant tail, and graceful Siamese-like body"),
    ("东方长毛猫", "an Oriental Longhair cat with fine silky coat, large ears, almond eyes, and long tubular body"),
    
    # 欧美品种
    ("美国短毛猫", "an American Shorthair cat with dense coat, broad chest, round face with sweet expression, and medium-sized muscular body"),
    ("挪威森林猫", "a Norwegian Forest cat with thick double coat, large tufted ears, long bushy tail, and sturdy body built for cold climate"),
    ("土耳其安哥拉", "a Turkish Angora cat with silky medium-long coat, elegant long body, pointed ears, and often odd-colored eyes"),
    ("阿比西尼亚猫", "an Abyssinian cat with ticked tabby coat, large almond eyes, distinctive wild appearance, and lithe muscular body"),
    
    # 短毛品种
    ("德文卷毛猫", "a Devon Rex cat with soft wavy coat, large low-set ears, high cheekbones, and slender body"),
    ("埃及猫", "an Egyptian Mau cat with spotted pattern, green almond eyes, worried expression, and athletic body"),
    ("新加坡猫", "a Singapura cat with sepia-toned ticked coat, large eyes, small size, and delicate appearance"),
    
    # 特殊品种
    ("重点色短毛猫", "a Colorpoint Shorthair cat with Siamese-like points in various colors, blue eyes, and elegant body"),
    ("异国短毛猫", "an Exotic Shorthair cat with dense plush coat, round face, snub nose, and cobby body like a Persian"),
    ("曼基康猫", "a Munchkin cat with normal-sized body and very short legs due to genetic mutation"),
    ("拉邦猫", "a LaPerm cat with curly coat, medium-sized body, and alert expression"),
    
    # 稀有品种
    ("库里安短尾猫", "a Kurilian Bobtail cat with naturally short pompom tail, strong body, and wild appearance"),
    ("塞尔凯克卷毛猫", "a Selkirk Rex cat with thick curly fur, round head, and sturdy body"),
    ("托基尼斯猫", "a Tonkinese cat with medium build, pointed pattern, aqua eyes, and muscular body")
]


# 生成文本输入
text_inputs = [f"a high quality photograph of {desc}, detailed, professional pet photography, 4k, sharp focus" 
               for _, desc in cat_types]
text = clip.tokenize(text_inputs).to(device)

# 处理图像
image = preprocess(Image.open("wideCat.png")).unsqueeze(0).to(device)

with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)
    
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

# 打印结果
print("\nTop predictions:\n")
sorted_results = sorted(zip(cat_types, probs[0]), key=lambda x: x[1], reverse=True)
for (chinese_name, _), prob in sorted_results[:10]:  # 显示前10个最可能的结果
    print(f"{chinese_name:>16s}: {100 * prob:.2f}%")


Top predictions:

             波斯猫: 24.68%
           异国短毛猫: 19.23%
          阿比西尼亚猫: 12.61%
            斯芬克斯: 9.82%
           东方长毛猫: 4.94%
            曼基康猫: 4.86%
            英短蓝猫: 4.64%
             折耳猫: 4.03%
             拉邦猫: 3.67%
            新加坡猫: 3.39%
